In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader,SubsetRandomSampler
from scipy import signal

import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from common import setup_dataflow,EEG,getepoch,do_plot,create_dataloader
import wandb
from mne.datasets import sample

In [2]:
# home directory + datasets folder
#path = "/root/EEG_Model/dataset/recorded_EEG/"
path = "/root/EEG_Model/dataset/finetune_EEG/"
#subject to run
#runs = [3,5,7,9]
runs = [4,6,8,10]
#runs = [7,8,9,10]
subjects = [9]
#recorded eeg class
eeg = EEG(path, subjects, runs)
raw=eeg.data_to_raw()

print("Raw done")
# apply filter 
raw=raw.notch_filter([50,75,100])
raw=raw.filter(8,14, method='fir', verbose=20)
print("Filter done")
#raw=eeg.raw_ica()
#eeg.create_epochs()

/root/EEG_Model/dataset/finetune_EEG/


/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S009/S009R04.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S009/S009R06.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S009/S009R08.fif) does not conform to MNE naming conventions. A

['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8', 'STIM MARKERS']
250.0
['C3', 'C4', 'STIM MARKERS']


/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S009/S009R08.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S009/S009R10.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )


['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8', 'STIM MARKERS']
250.0
['C3', 'C4', 'STIM MARKERS']
Raw done
Filtering raw data in 4 contiguous segments
Setting up band-pass filter from 8 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filter done


In [3]:
epochs=eeg.epochs(raw,tmin=0,tmax=7,baseline=(0,2))
#X = X[:, :,np.newaxis,:]
X, y = eeg.get_X_y(epochs)
print(X.shape)
print(y)

240 events found
Event IDs: [1 2 4]
Not setting metadata
Not setting metadata
80 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 80 events and 1751 original time points ...
4 bad epochs dropped
(76, 2, 1751)
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0]


/root/EEG_Model/common.py:280: RuntimeWarning: No matching events found for 3 (event id 3)
  epochs = mne.Epochs(


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)
print('Train size',X_train.shape, y_train.shape)
print('Test size',X_test.shape, y_test.shape)


Train size (53, 2, 1751) (53,)
Test size (23, 2, 1751) (23,)


In [5]:
batch_size = 1751

train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)

num_step =math.ceil(len(train_loader.dataset) / batch_size)

In [6]:
#wand setup
wandb.login()
wand = wandb.init(
      # Set the project where this run will be logged
      project="Motor-Imagery-New", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"CNN_S09_MI", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0000001,
      "architecture": "CNN2D",
      "dataset": "Recorded",
      "epochs": 100000,
      "weightname":"Sunsun_S09_new_MI",
      "num_step_per_epoch" : num_step, 
      }
    )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: newturno. Use `wandb login --relogin` to force relogin


In [7]:
from model import ConvNet,CNN2D
from torchsummary import summary
net = ConvNet().cuda()

#summary(net, (2, 641),32)

# Training section

In [8]:
from common import train
config = wand.config
optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()
device = 'cuda'

train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    loader_train = train_loader,
    loader_test = test_loader,
    vail_loader = None,
    optimizer = optimizer  ,
    criterion = criterion ,
    device = 'cuda',
    wand = wand
)


wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

Epoch 1/100000, Tr Loss: 0.7729, Tr Acc: 41.5094, Val Loss: 0.6931, Val Acc: 52.1739
Epoch 101/100000, Tr Loss: 0.7139, Tr Acc: 52.8302, Val Loss: 0.6985, Val Acc: 47.8261
Epoch 201/100000, Tr Loss: 0.7331, Tr Acc: 43.3962, Val Loss: 0.7034, Val Acc: 43.4783
Epoch 301/100000, Tr Loss: 0.7070, Tr Acc: 45.2830, Val Loss: 0.7019, Val Acc: 34.7826
Epoch 401/100000, Tr Loss: 0.6992, Tr Acc: 60.3774, Val Loss: 0.7053, Val Acc: 43.4783
Epoch 501/100000, Tr Loss: 0.6992, Tr Acc: 50.9434, Val Loss: 0.7059, Val Acc: 39.1304
Epoch 601/100000, Tr Loss: 0.6213, Tr Acc: 66.0377, Val Loss: 0.7029, Val Acc: 34.7826
Epoch 701/100000, Tr Loss: 0.5913, Tr Acc: 71.6981, Val Loss: 0.7076, Val Acc: 52.1739
Epoch 801/100000, Tr Loss: 0.6815, Tr Acc: 56.6038, Val Loss: 0.7018, Val Acc: 52.1739
Epoch 901/100000, Tr Loss: 0.6009, Tr Acc: 67.9245, Val Loss: 0.7018, Val Acc: 52.1739
Epoch 1001/100000, Tr Loss: 0.6031, Tr Acc: 69.8113, Val Loss: 0.6997, Val Acc: 56.5217
Epoch 1101/100000, Tr Loss: 0.5888, Tr Acc: 

KeyboardInterrupt: 

# Finetune data
You need to freeze layer

In [ ]:
net.load_state_dict(torch.load('./save_weight/Krittin_S54_MI/0.6929_Krittin_S54_MI_0.6929_52.1739'))

In [ ]:
parms = net.state_dict()
parms.keys()

### Freeze layer 1 and 2

In [ ]:
for name,param in net.named_parameters():
    if param.requires_grad and 'layer1' in name:
        param.requires_grad = False
    if param.requires_grad and 'layer2' in name:
        param.requires_grad = False

In [ ]:
# home directory + datasets folder
path = "/root/EEG_Model/dataset/finetune_EEG/"
#subject to run
runs = [4,6,8,10]
subjects = [8]
#recorded eeg class
eeg = EEG(path, subjects, runs)
raw=eeg.data_to_raw()

print("Raw done")
# apply filter 
raw=raw.notch_filter([50,75,100])
raw=raw.filter( 8,14, method='fir', verbose=20)
print("Filter done")

epochs=eeg.epochs(raw,tmin=0,tmax=7,baseline=(0,2))
#X = X[:, :,np.newaxis,:]
X, y = eeg.get_X_y(epochs)
print(X.shape)
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

print('train size',X_train.shape, y_train.shape)
print('Test size',X_test.shape, y_test.shape)

In [ ]:
batch_size = 1751

train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)

num_step =math.ceil(len(train_loader.dataset) / batch_size)

In [ ]:
import wandb
wandb.login()


wand = wandb.init(
        
      # Set the project where this run will be logged
      project="Motor-Imagery", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"CNN_S08_New_MI", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0000001,
      "architecture": "CNN",
      "dataset": "Nutapol T.",
      "epochs": 300000,
      "weightname":"Krittin_S08_Finetune_MI",
      "num_step_per_epoch" : num_step, 
        
      }
    )

config = wand.config
print(config.num_step_per_epoch)

In [ ]:
from common import train
config = wand.config
optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()
device = 'cuda'

train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    loader_train = train_loader,
    loader_test = test_loader,
    vail_loader = None,
    optimizer = optimizer  ,
    criterion = criterion ,
    device = 'cuda',
    wand = wand
)

wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

# Validate Finetune

In [ ]:
del net

In [ ]:
from model import ConvNet,CNN2D
from torchsummary import summary
net = ConvNet().cuda()
net.load_state_dict(torch.load('./save_weight/Sunsun_findtune_MI/0.4597_Sunsun_findtune_MI_0.4597_83.3333'))

In [ ]:
# home directory + datasets folder
path = "/root/EEG_Model/dataset/finetune_EEG/"
#subject to run
runs = [4]
subjects = [20]
#recorded eeg class
eeg = EEG(path, subjects, runs)
raw=eeg.data_to_raw()

print("Raw done")
# apply filter 
raw=raw.notch_filter([50,75,100])
raw=raw.filter( 8,14, method='fir', verbose=20)
print("Filter done")

epochs=eeg.epochs(raw,tmin=0,tmax=7,baseline=(0,2))
#X = X[:, :,np.newaxis,:]
X, y = eeg.get_X_y(epochs)
print(X.shape)
print(y)

In [ ]:
batch_size = 1751
test_loader = create_dataloader(X, y, batch_size=batch_size)

num_step =math.ceil(len(test_loader.dataset) / batch_size)

In [ ]:
# Train the model
num_epochs = 1

train_loss = []
valid_loss = []
train_accuracy = []
valid_accuracy = []
old_acc = 0
for epoch in range(num_epochs):

    loss = 0.0
    correct = 0
    iterations = 0
    net.eval()
    for i, (items, classes) in enumerate(test_loader):
        items = Variable(items)
        classes = Variable(classes)
        
        if cuda.is_available():
            items = items.cuda(0)
            classes = classes.cuda(0)
        
        outputs = net(items)
        #loss += criterion(outputs, classes).item()
        
        _, predicted = torch.max(outputs.data, 1)
        
        for i in range(predicted.shape[0]):
            
            if predicted[i] == classes.data[i]:
                correct += 1
            
        #correct += (predicted == classes.data).sum()
        
        iterations += 1

    #valid_loss.append(loss/iterations)
    #correct_scalar = np.array([correct.clone().cpu()])[0]
    valid_accuracy.append(correct / len(test_loader.dataset) * 100.0)
print (' Val Acc: %.4f'
                   %(valid_accuracy[-1]))

In [ ]:
print(predicted)
print(classes.data.cpu().numpy())
y_true = classes.data.cpu().numpy()
y_pred = y_pred=predicted.cpu().numpy()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
confusion_matrix(y_true, y_pred)

In [ ]:

ConfusionMatrixDisplay.from_predictions(y_true, y_pred)